<a href="https://colab.research.google.com/github/yukinaga/minnano_kaggle/blob/main/section_4/01_tabular_playground_series_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tabular Playground Series - Jan 2022に挑戦しよう
Kaggleグッズの3カ国にわたる販売数を予測しましょう。 

## データの読み込み
Tabular Playground Series - Jan 2022のデータを読み込みます。  
以下のページからデータをダウロードして、「train.csv」「test.csv」をノートブック環境にアップします。  
https://www.kaggle.com/c/titanic/data

訓練データには販売数を表す"num_sold"の列がありますが、テストデータにはありません。  
訓練済みのモデルに、テストデータを入力して判定した結果を提出することになります。  


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import lightgbm as lgb


train_data = pd.read_csv("train.csv")  # 訓練データ
test_data = pd.read_csv("test.csv") # テストデータ

test_id = test_data["row_id"]  # 結果の提出時に使用

train_data.head()

`info()`によりデータの全体像を確認することができます。  
Non-Null Countにより欠損していないデータの数が確認できるので、データの欠損が存在するかどうか確認しておきます。  

In [ ]:
train_data.info()
test_data.info()

## データの加工
データを加工し、使用可能な形にします。

In [ ]:
data = pd.concat([train_data, test_data], sort=False)  # テストデータ、訓練データを結合

# カテゴリデータの変換
data["country"].replace(["Finland", "Norway", "Sweden"], [0, 1, 2], inplace=True)
data["store"].replace(["KaggleMart", "KaggleRama"], [0, 1], inplace=True)
data["product"].replace(["Kaggle Mug", "Kaggle Hat", "Kaggle Sticker"], [0, 1, 2], inplace=True)

# 新しい特徴量の作成
data["date"] = pd.to_datetime(data["date"])
data["day"] = data.date.dt.day
data["month"] = data.date.dt.month
data["year"] = data.date.dt.year
data["weekday"] = data.date.dt.weekday

# 不要な特徴量の削除
data.drop(["row_id", "date"], axis=1, inplace=True)

# 入力と正解の作成
train_data = data[:len(train_data)]
test_data = data[len(train_data):]
t = train_data["num_sold"]  # 正解
x_train = train_data.drop("num_sold", axis=1)  # 訓練時の入力
x_test = test_data.drop("num_sold", axis=1)  # テスト時の入力

x_train.head()

## データの分割
訓練用データを訓練用と検証用に分割します。

In [ ]:
x_train, x_valid, t_train, t_valid = train_test_split(x_train, t, test_size=0.3)

## モデルの訓練
機械学習のアルゴリズムには、決定木をベースにした「LightGBM」を使います。  
今回は販売数を予測するので、分類問題ではなく回帰問題です。  
LightGBMのライブラリでは、「LGBMRegressor」により回帰問題を扱うことができます。  
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html


In [ ]:
categorical_features = ["country", "store", "product", "month", "year", "weekday"]

# モデルの学習
model = lgb.LGBMRegressor() # モデルのインスタンスの作成

# モデルの訓練
model = model.fit(x_train, t_train, eval_set=[(x_train, t_train), (x_valid, t_valid)],
                    categorical_feature=categorical_features)

# 誤差の推移を表示
lgb.plot_metric(model)

## 提出用のデータを作成
訓練用のモデル、テストデータを使って予測を行います。

In [ ]:
# 訓練済みのモデルを使って予測
y_test = model.predict(x_test)
print(y_test)


提出量データの形式を整え、CSVファイルに保存します。

In [ ]:
# 形式を整える
survived_test = pd.Series(y_test, name="num_sold")
subm_data = pd.concat([test_id, survived_test], axis=1)

# 提出用のcsvファイルを保存
subm_data.to_csv("submission_tps_jan2022.csv", index=False)

subm_data